In [1]:
import pandas as pd
import sqlite3

In [2]:
# SQLite veritabanı bağlantısını oluşturun
conn = sqlite3.connect('salaries.db')  

In [3]:
# Veri setini okuyun ve SQLite veritabanına aktarın
data = pd.read_csv('/kaggle/input/data-jobs-salaries/salaries.csv')
data.to_sql('salaries', conn, if_exists='replace', index=False)

8545

In [5]:
# SQL sorguları

#Tüm tabloya göz atalım
query = "Select * From salaries"
tables = pd.read_sql(query, conn)
tables

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Data Engineer,166000,USD,166000,US,0,US,M
1,2023,SE,FT,Data Engineer,73100,USD,73100,US,0,US,M
2,2023,MI,FT,Business Intelligence Developer,140000,USD,140000,US,100,US,M
3,2023,MI,FT,Business Intelligence Developer,110000,USD,110000,US,100,US,M
4,2023,MI,FT,Data Scientist,130900,USD,130900,US,0,US,M
...,...,...,...,...,...,...,...,...,...,...,...
8540,2020,SE,FT,Data Scientist,412000,USD,412000,US,100,US,L
8541,2021,MI,FT,Principal Data Scientist,151000,USD,151000,US,100,US,L
8542,2020,EN,FT,Data Scientist,105000,USD,105000,US,100,US,S
8543,2020,EN,CT,Business Data Analyst,100000,USD,100000,US,100,US,L


In [9]:
#Ortalama dolar cinsinden maaş
query2 = "Select AVG(salary_in_usd) AS average_salary_in_usd From salaries"
tables2 = pd.read_sql(query2, conn)
tables2

,average_salary_in_usd
0,149365.846811


In [10]:
# En yüksek maaş (dolar cinsinden seçtim çünkü diğer para birimlerinde çok yüksek maşlar var fakat dolar cinsine çevirince pek birşey etmiyor:))

query3 = "Select max(salary_in_usd) AS highest_salary From salaries"
tables3 = pd.read_sql(query3, conn)
tables3

,highest_salary
0,450000


In [11]:
# En yüksek maaşlı 10 iş ünvanı
query4 = "Select experience_level,job_title, salary_in_usd From salaries order by salary_in_usd DESC LIMIT 10;"
tables4 = pd.read_sql(query4, conn)
tables4

,experience_level,job_title,salary_in_usd
0,MI,Research Scientist,450000
1,MI,Data Analyst,430967
2,MI,Analytics Engineer,430640
3,MI,Applied Machine Learning Scientist,423000
4,SE,AI Scientist,417937
5,EX,Principal Data Scientist,416000
6,SE,Data Scientist,412000
7,MI,Research Scientist,405000
8,SE,Data Analytics Lead,405000
9,SE,Analytics Engineering Manager,399880


In [13]:
# Deneyim seviyesine göre ortalama maaşlar
query5 = "Select experience_level,avg(salary_in_usd) as average_salary From salaries group by experience_level;"
tables5 = pd.read_sql(query5, conn)
tables5

,experience_level,average_salary
0,EN,86323.428571
1,EX,190135.566540
2,MI,114546.295346
3,SE,161859.455078


In [14]:
# Remote, hybrid ve remote olmayan olarak iş sayısı
query6 = "Select remote_ratio, count(*) AS job_count from salaries group by remote_ratio;"
tables6 = pd.read_sql(query6, conn)
tables6

,remote_ratio,job_count
0,0,5087
1,50,218
2,100,3240


In [15]:
# Her istihdam türü için ortalama maaş
query7 = "Select employment_type, avg(salary_in_usd) as average_salary from salaries group by employment_type;"
tables7 = pd.read_sql(query7, conn)
tables7

,employment_type,average_salary
0,CT,120837.666667
1,FL,54734.363636
2,FT,149697.437845
3,PT,52053.153846


In [16]:
# Şirket büyüklüklerine göre ortalama maaş
query8 = "Select company_size, avg(salary_in_usd) as average_salary from salaries group by company_size;"
tables8 = pd.read_sql(query8, conn)
tables8

,company_size,average_salary
0,L,133720.521918
1,M,152187.006276
2,S,88556.622754


In [17]:
# İstihdam türlerinin dağılımı
query9 = "Select employment_type, count(*) as job_count from salaries group by employment_type;"
tables9 = pd.read_sql(query9, conn)
tables9

,employment_type,job_count
0,CT,18
1,FL,11
2,FT,8503
3,PT,13


In [18]:
#Yıllara Göre Maaş Trendleri
query10 = "Select work_year, avg(salary_in_usd) as average_salary from salaries group by work_year order by work_year;"
tables10 = pd.read_sql(query10, conn)
tables10

,work_year,average_salary
0,2020,102250.866667
1,2021,99922.073394
2,2022,134449.138098
3,2023,155264.930617


In [22]:
#Şirket büyüklüğü ve deneyim seviyesine göre ortalama maaşlar
query11 = "SELECT company_size, experience_level, AVG(salary_in_usd) AS average_salary FROM salaries GROUP BY company_size, experience_level;"
tables11 = pd.read_sql(query11, conn)
tables11

,company_size,experience_level,average_salary
0,L,EN,74840.783784
1,L,EX,184204.105263
2,L,MI,100100.784884
3,L,SE,160260.376168
4,M,EN,93189.062706
5,M,EX,191323.716102
6,M,MI,117984.433818
7,M,SE,162422.520826
8,S,EN,69537.729167
9,S,EX,169172.375000


In [23]:
#Yıllara göre uzaktan çalışma oranı
query12 = "Select work_year, avg(remote_ratio) as average_remote_ratio from salaries group by work_year;"
tables12 = pd.read_sql(query12, conn)
tables12

,work_year,average_remote_ratio
0,2020,62.000000
1,2021,69.954128
2,2022,55.208964
3,2023,33.911529


In [24]:
#  İş Unvanı ve Şirket Büyüklüğüne Göre ortalama Maaş
query13 = "Select job_title, company_size, avg(salary_in_usd) as average_salary from salaries group by job_title, company_size;"
tables13 = pd.read_sql(query13, conn)
tables13

,job_title,company_size,average_salary
0,AI Architect,L,207968.000000
1,AI Architect,M,258800.000000
2,AI Developer,L,172968.000000
3,AI Developer,M,131242.181818
4,AI Developer,S,147246.000000
...,...,...,...
220,Staff Data Analyst,L,44753.000000
221,Staff Data Analyst,M,15000.000000
222,Staff Data Analyst,S,179998.000000
223,Staff Data Scientist,M,134500.000000


In [25]:
# Deneyim Düzeyine Göre Ortalama Veri Bilimci Maaş
query14 = "Select experience_level, avg(salary_in_usd) as average_salary from salaries where job_title = 'Data Scientist' group by experience_level;"
tables14 = pd.read_sql(query14, conn)
tables14

,experience_level,average_salary
0,EN,84620.741176
1,EX,186803.724138
2,MI,111937.920962
3,SE,169003.290022


In [26]:
# Deneyim düzeyine göre ortalama Makine öğrenmesi mühendisi maaşı
query15 = "Select experience_level, avg(salary_in_usd) as average_salary from salaries where job_title = 'Machine Learning Engineer' group by experience_level;"
tables15 = pd.read_sql(query15, conn)
tables15

,experience_level,average_salary
0,EN,95418.428571
1,EX,190950.000000
2,MI,144630.045455
3,SE,191713.076923


In [27]:
# Deneyim Düzeyine Göre Ortalama Veri Analisti Maaş
query16 = "Select experience_level, avg(salary_in_usd) as average_salary from salaries where job_title = 'Data Analyst' group by experience_level;"
tables16 = pd.read_sql(query16, conn)
tables16

,experience_level,average_salary
0,EN,68545.405941
1,EX,107000.000000
2,MI,97925.854103
3,SE,119343.893285


In [28]:
# Deneyim Düzeyine Göre Ortalama Veri Mühendisi Maaş
query17 = "Select experience_level, avg(salary_in_usd) as average_salary from salaries where job_title = 'Data Engineer' group by experience_level;"
tables17 = pd.read_sql(query17, conn)
tables17

,experience_level,average_salary
0,EN,93039.782609
1,EX,190115.412371
2,MI,113569.884236
3,SE,156383.327477


In [29]:
# Yıllara göre maaş trendi (Veri Bilimci)
query18 = "Select work_year, avg(salary_in_usd) as average_salary from salaries where job_title = 'Data Scientist' group by work_year order by work_year;"
tables18 = pd.read_sql(query18, conn)
tables18

,work_year,average_salary
0,2020,85970.523810
1,2021,79650.425000
2,2022,136902.710396
3,2023,165270.439610


In [30]:
# Yıllara göre maaş trendi (Makine öğrenmesi mühendisi)
query19 = "Select work_year, avg(salary_in_usd) as average_salary from salaries where job_title = 'Machine Learning Engineer' group by work_year order by work_year;"
tables19 = pd.read_sql(query19, conn)
tables19

,work_year,average_salary
0,2020,145904.500000
1,2021,74611.222222
2,2022,150564.361111
3,2023,191083.704545


In [31]:
#Yıllara göre maaş trendi (Veri Analisti)
query20 = "Select work_year, avg(salary_in_usd) as average_salary from salaries where job_title = 'Data Analyst' group by work_year order by work_year;"
tables20 = pd.read_sql(query20, conn)
tables20

,work_year,average_salary
0,2020,60911.166667
1,2021,78258.500000
2,2022,108658.286765
3,2023,110919.885246


In [32]:
# Yıllara göre maaş trendi (Veri Mühendisi)
query21 = "Select work_year, avg(salary_in_usd) as average_salary from salaries where job_title = 'Data Engineer' group by work_year order by work_year;"
tables21 = pd.read_sql(query21, conn)
tables21

,work_year,average_salary
0,2020,85301.384615
1,2021,96143.081081
2,2022,139589.889344
3,2023,150579.476351


In [36]:
# Veri bilimci için Şirket Büyüklüğüne ve Deneyim Düzeyine Göre Ortalama Maaş
query22 = "Select company_size, experience_level, avg(salary_in_usd) as average_salary from salaries where job_title = 'Data scientist' group by company_size, experience_level;"
tables22 = pd.read_sql(query22, conn)
tables22

,company_size,experience_level,average_salary


In [37]:
#  Makine Öğrenmesi Mühendisi için Şirket Büyüklüğüne ve Deneyim Düzeyine Göre Ortalama Maaş
query23 = "Select company_size, experience_level, avg(salary_in_usd) as average_salary from salaries where job_title = 'Machine Learning Engineer' group by company_size, experience_level;"
tables23 = pd.read_sql(query23, conn)
tables23

,company_size,experience_level,average_salary
0,L,EN,102618.600000
1,L,MI,109651.857143
2,L,SE,155934.178571
3,M,EN,102718.500000
4,M,EX,190950.000000
5,M,MI,154849.390805
6,M,SE,193970.737288
7,S,EN,83618.250000
8,S,MI,100253.555556
9,S,SE,72390.200000


In [38]:
# Veri Analisti için Şirket Büyüklüğüne ve Deneyim Düzeyine Göre Ortalama Maaş
query24 = "Select company_size, experience_level, avg(salary_in_usd) as average_salary from salaries where job_title = 'Data Analyst' group by company_size, experience_level;"
tables24 = pd.read_sql(query24, conn)
tables24

,company_size,experience_level,average_salary
0,L,EN,55255.714286
1,L,MI,74367.000000
2,L,SE,144113.100000
3,M,EN,72540.432432
4,M,EX,107000.000000
5,M,MI,99650.754839
6,M,SE,119369.935049
7,S,EN,65787.333333
8,S,MI,45333.333333
9,S,SE,85726.125000


In [40]:
# Veri bilimcisi, veri analisti ve makine öğrenimi mühendisi olmak üzere üç belirli meslek için yıllara ve deneyim düzeyine göre ortalama maaşlar

query25 = "Select work_year, experience_level, job_title,  avg(salary_in_usd) as average_salary from salaries where job_title IN ('Data Scientist' ,'Machine Learning Engineer' ,'Data Analyst') group by work_year, experience_level, job_title order by work_year, experience_level, job_title;"
tables25 = pd.read_sql(query25, conn)
tables25

,work_year,experience_level,job_title,average_salary
0,2020,EN,Data Analyst,58427.000000
1,2020,EN,Data Scientist,54983.333333
2,2020,EN,Machine Learning Engineer,194000.000000
3,2020,MI,Data Analyst,65879.500000
4,2020,MI,Data Scientist,71256.000000
5,2020,SE,Data Scientist,172916.250000
6,2020,SE,Machine Learning Engineer,97809.000000
7,2021,EN,Data Analyst,63521.666667
8,2021,EN,Data Scientist,63604.333333
9,2021,EN,Machine Learning Engineer,59611.166667
